# Kinematic Data Augmention

This notebook demonstrates a data augmentation process for kinematic data, including the estimation of additional joint positions, angles, and distances, using geometric methods for both relative and absolute values, with the assumption that the performer is in an upright position.

### Required Modules

In [ ]:
import os
import cv2
import math
import pandas as pd
import numpy as np

## Relative and Absolute Values

Converting kinematic data to absolute values can be useful for measuring the movement of a performer in physical space, while using relative data is better for capturing movement characteristics independent of physical space, such as the relative positions of the limbs. For our research, we will keep both absolute and relative kinematic data.

In [ ]:
h, w = 360, 640
kd_rel = pd.read_csv('./data/processed/contemporary_001.csv')
kd_abs = kd_rel.copy()

kd_abs.loc[:, kd_abs.columns.str.endswith('_x')] *= w
kd_abs.loc[:, kd_abs.columns.str.endswith('_y')] *= h
kd_abs.loc[:, kd_abs.columns.str.endswith('_z')] = h - kd_abs.loc[:, kd_abs.columns.str.endswith('_z')]

## Augmenting Joint Positions

MediaPipe's current implementation involves using 32 landmarks to identify joints (as shown in a reference picture). In order to gain a more comprehensive understanding of body alignment and posture during performances, we have estimated additional joint positions, which can be helpful in achieving this goal.

![Image](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

### Forehead Joint

This computation estimates the position of the forehead joint using the average position of the outer corners of both eyes in both relative and absolute kinematic data.

In [ ]:
kd_rel.loc[:, 'forehead_x'] = (kd_rel['left_eye_outer_x'] + kd_rel['right_eye_outer_x']) / 2
kd_rel.loc[:, 'forehead_y'] = (kd_rel['left_eye_outer_y'] + kd_rel['right_eye_outer_y']) / 2
kd_rel.loc[:, 'forehead_z'] = (kd_rel['left_eye_outer_z'] + kd_rel['right_eye_outer_z']) / 2

kd_abs.loc[:, 'forehead_x'] = (kd_abs['left_eye_outer_x'] + kd_abs['right_eye_outer_x']) / 2
kd_abs.loc[:, 'forehead_y'] = (kd_abs['left_eye_outer_y'] + kd_abs['right_eye_outer_y']) / 2
kd_abs.loc[:, 'forehead_z'] = (kd_abs['left_eye_outer_z'] + kd_abs['right_eye_outer_z']) / 2

### Torso Joint

This computation estimates the location of the torso joint by averaging the positions of the left and right shoulders and left and right hips.

In [ ]:
kd_rel.loc[:, 'torso_x'] = (kd_rel['left_shoulder_x'] + kd_rel['right_shoulder_x'] + kd_rel['left_hip_x'] + kd_rel['right_hip_x']) / 4
kd_rel.loc[:, 'torso_y'] = (kd_rel['left_shoulder_y'] + kd_rel['right_shoulder_y'] + kd_rel['left_hip_y'] + kd_rel['right_hip_y']) / 4
kd_rel.loc[:, 'torso_z'] = (kd_rel['left_shoulder_z'] + kd_rel['right_shoulder_z'] + kd_rel['left_hip_z'] + kd_rel['right_hip_z']) / 4

kd_abs.loc[:, 'torso_x'] = (kd_abs['left_shoulder_x'] + kd_abs['right_shoulder_x'] + kd_abs['left_hip_x'] + kd_abs['right_hip_x']) / 4
kd_abs.loc[:, 'torso_y'] = (kd_abs['left_shoulder_y'] + kd_abs['right_shoulder_y'] + kd_abs['left_hip_y'] + kd_abs['right_hip_y']) / 4
kd_abs.loc[:, 'torso_z'] = (kd_abs['left_shoulder_z'] + kd_abs['right_shoulder_z'] + kd_abs['left_hip_z'] + kd_abs['right_hip_z']) / 4

## Augmenting Joint Angles and Magnitude
Augmenting joint angles and distances using dot product and Euclidean distance can be used to extract body alignment and posture during a performance. The joint pairs defined represent connections between limbs in the human body.

### Defining Connected Joints
The joint pairs defined represent connections between limbs in the human body.

In [ ]:
joint_pairs = [('right_shoulder', 'right_elbow'), ('right_elbow', 'right_wrist'), ('right_hip', 'right_knee'), ('right_knee', 'right_ankle'), ('right_ankle', 'right_foot_index'), ('left_shoulder', 'left_elbow'), ('left_elbow', 'left_wrist'), ('left_hip', 'left_knee'), ('left_knee', 'left_ankle'), ('left_ankle', 'left_foot_index'), ('forehead', 'torso')]

### Computing Angles and Magnitude

In [ ]:
for joint in joint_pairs:
    angles = []
    for i in range(kd_rel.shape[0]):
        joint1 = np.array([kd_rel[f"{joint[0]}_x"].iloc[i], kd_rel[f"{joint[0]}_y"].iloc[i], kd_rel[f"{joint[0]}_z"].iloc[i]])
        joint2 = np.array([kd_rel[f"{joint[1]}_x"].iloc[i], kd_rel[f"{joint[1]}_y"].iloc[i], kd_rel[f"{joint[1]}_z"].iloc[i]])     
        dot_product = np.dot(joint1, joint2)
        mag1 = np.linalg.norm(joint1)
        mag2 = np.linalg.norm(joint2)
        dist = np.linalg.norm(joint1 - joint2)
        angle = np.degrees(np.arccos(dot_product / (mag1 * mag2)))
        angles.append(angle)
    kd_rel[f"a_{joint[0]}_{joint[1]}"] = angles
    kd_rel[f"d_{joint[0]}_{joint[1]}"] = dist
    
for joint in joint_pairs:
    angles = []
    for i in range(kd_abs.shape[0]):
        joint1 = np.array([kd_abs[f"{joint[0]}_x"].iloc[i], kd_abs[f"{joint[0]}_y"].iloc[i], kd_abs[f"{joint[0]}_z"].iloc[i]])
        joint2 = np.array([kd_abs[f"{joint[1]}_x"].iloc[i], kd_abs[f"{joint[1]}_y"].iloc[i], kd_abs[f"{joint[1]}_z"].iloc[i]])    
        dot_product = np.dot(joint1, joint2)
        mag1 = np.linalg.norm(joint1)
        mag2 = np.linalg.norm(joint2)
        angle = np.degrees(np.arccos(dot_product / (mag1 * mag2)))
        angles.append(angle)
    kd_abs[f"a_{joint[0]}_{joint[1]}"] = angles
    kd_abs[f"d_{joint[0]}_{joint[1]}"] = dist

### Save Processed DataFrames

In [ ]:
kd_abs.to_csv('./data/processed/contemporary_001_abs.csv', index=False)
kd_rel.to_csv('./data/processed/contemporary_001_rel.csv', index=False)

## Batch Processing

For more details about the data augmentation processing, please refer to the code in [/src/data/augmentation.py](https://github.com/kayesokua/gestures/blob/main/src/data/augmentation.py)

In [ ]:
from src.data.augmentation import batch_data_augmentation

batch_data_augmentation("./data/processed")

## Summary

* The data augmentation process enriches kinematic data by adding new joint positions, angles, and distances which may extract further insights on gestures.
* The analysis can be performed on both absolute and relative values for the available landmarks, which might extract insight on spatial patterns.
    * Augmented Kinematic data using absolute values: `data/processed/{category_i}_abs.csv`
    * Augmented Kinematic data using absolute values: `data/processed/{category_i}_rel.csv`